In [ ]:
import os
import sys
sys.path.append('E:/CURRENT PROJECTS/J1_check in-situ para/code/')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from yellowbrick.target.feature_correlation import feature_correlation
from sklearn.metrics import (r2_score, mean_absolute_error, mean_squared_error,explained_variance_score,
                             accuracy_score, f1_score, recall_score, precision_score)

from yellowbrick.features import rank1d, rank2d
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.model_selection import RandomizedSearchCV
from yellowbrick.model_selection import feature_importances
import seaborn as sns
from scipy.stats import kurtosis,skew

In [ ]:
def sumdata(df,key,var): #key variables
    global sumdf
    dict = {k: v for k, v in zip(key, var)}
    sumdf=pd.DataFrame({'Variables': [],'Description':[], 'Min':[], 'Max':[], 'Mean':[],'STD':[], 'Ske':[],'Kur':[]})
    for key in dict:
        v=dict[key]
        ab=key
        min=df[key].min()
        max=df[key].max()
        mean=('%.3f' % df[key].mean())
        std=('%.3f ' % df[key].std())
        ske=('%.3f ' % skew(df[key], axis = 1, bias = True))
        kur=('%.3f ' % kurtosis(df[key], axis = 0, fisher = True, bias = True))
        sumdf = sumdf.append({'Variables': ab, 'Description': v, 'Min': min, 'Max': max, 'Mean':mean,'STD':std,'Ske':ske,'Kur':kur},ignore_index=True)
    print(sumdf)
    return sumdf

In [ ]:
import time
def bootstrap_nmodel_ktarget(df,features,targets,n_iterations,n_size,scale,nmodels): #model in list, not dict
    # bootstrap sampling
    global result
    result = pd.DataFrame({'Target':[],'Iteration':[],'Models': [],'runtime_s':[], 'R2 train': [], 'RMSE train': [], 'R2 test': [],'RMSE test': []})
    for k in targets:
        for i in range(n_iterations):
            print(i)
        # prepare train and test sets
            train, test = train_test_split(df, train_size=n_size,random_state=i)
            X_train = scale.fit_transform(train[features])
            y_train = train[k]
            X_test = scale.transform(test[features])
            y_test = test[k]
            # fit model
            for m, model in nmodels:
                start_time = time.time()
                model.fit(X_train, y_train)
                pred1 = model.predict(X_train)
                pred2 = model.predict(X_test)
                end_time = time.time()
                runtime= end_time - start_time
                r1 = ('%.3f' % r2_score(y_train, pred1))
                r2 = ('%.3f' % r2_score(y_test, pred2))
                rmse1 = ('%.3f' % mean_squared_error(y_train, pred1, squared=False))
                rmse2 = ('%.3f' % mean_squared_error(y_test, pred2, squared=False))
                result = result.append(
                    {'Target':k,'Iteration': i, 'Models': m,'runtime_s':runtime, 'R2 train': r1, 'RMSE train': rmse1, 'R2 test': r2,
                     'RMSE test': rmse2}, ignore_index=True)
    print('done!\n', result)
    return result


In [ ]:
def tune_RFR(model,df,features,target,n_iter,n_size,cv,scaler, rd_state):
    global result, bestmodel
    n_estimators = [int(x) for x in np.linspace(start=500, stop=2000, num=100)]
    max_depth = [int(x) for x in np.linspace(5,15, num=10)]
    min_samples_split = [int(x) for x in np.linspace(4, 20, num=8)]
    min_samples_leaf = [int(x) for x in np.linspace(4, 20, num=8)]
    max_features=[int(x) for x in range(5, len(features)+1,1)]
    #oob_score=[True]
    #max_samples=[int(x) for x in np.linspace(20, 500, num=20)]

    random_grid = {'n_estimators': n_estimators,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'max_features':max_features
                                                         }
    print(random_grid)
    X = df[features]
    y = df[target]
    X_train, X_test, y_train, y_test = getsplit_rd(X, y,n_size,scaler,rd_state)
    search = RandomizedSearchCV(estimator=model, param_distributions=random_grid, n_iter=n_iter, cv=cv,
                                random_state=rd_state, verbose=1, n_jobs=-1)

    search.fit(X_train, y_train)
    bestmodel = search.best_estimator_
    result=fitted_model_val(bestmodel, X_train, X_test, y_train, y_test)
    print(' bestmodel\n',search.best_params_)
    return result, bestmodel


In [ ]:
#uncertainty
def df_actual_Qpred(model,quantiles,X_test,y_test,path,name):
    '''create dataframe of quantile prediction'''
    pred_Q = pd.DataFrame()
    for pred in model.estimators_:
        temp = pd.Series(pred.predict(X_test).round(2))
        pred_Q = pd.concat([pred_Q, temp], axis=1)
    print(pred_Q.head())
    RF_actual_pred = pd.DataFrame()
    for q in quantiles:
        s = pred_Q.quantile(q=q, axis=1)
        RF_actual_pred = pd.concat([RF_actual_pred, s], axis=1, sort=False)
    RF_actual_pred.columns = quantiles
    RF_actual_pred['actual'] = y_test.to_list()
    RF_actual_pred['interval'] = RF_actual_pred[np.max(quantiles)] - RF_actual_pred[np.min(quantiles)]
    #RF_actual_pred = RF_actual_pred.sort_values('interval')
    RF_actual_pred = RF_actual_pred.round(2)
    print(RF_actual_pred.head())
    RF_actual_pred.to_csv(path + str(name)+ '_RF_pred.csv')
    return RF_actual_pred

def correctPcnt(df_actual_Qpred,Qlow,Qhigh):
    correct = 0
    for i in range(df_actual_Qpred.shape[0]):
        if df_actual_Qpred.loc[i, Qlow] <= df_actual_Qpred.loc[i, 'actual'] <= df_actual_Qpred.loc[i, Qhigh]:
            correct += 1
    print('Correct percentage: %.2f' % (correct / len(df_actual_Qpred)*100))

def showIntervals(df,df_show,Qlow,Qhigh,path,name):
    '''create plot of ytrue, y quantile of df_show for 100 st samples, score of all test
    '''
    plt.rcParams['axes.labelsize'] = 10
    plt.rcParams['axes.titlesize'] = 10
    plt.rcParams['legend.fontsize'] = 10
    fig, ax = plt.subplots(figsize=(4,5))

    ax.plot(df_show['actual'], 'go', markersize=3, label='Actual',color='b')
    ax.plot(df_show['0.5'],linewidth=0.8,  label='Median prediction',color='k')
    ax.fill_between(
        np.arange(df_show.shape[0]), df_show[Qlow], df_show[Qhigh], alpha=0.5, color="r",
        label="90% prediction interval")
    ax.set_xlabel("first 100 samples")
    ax.set_ylabel("Concentration (mg/l)")
    plt.xlim([0, len(df_show)+2])
    plt.ylim([0, (df_show['actual'].max()*1.1)])
    plt.legend()
    plt.title(' $R^2$= %.2f'% r2_score(df['actual'] , df['0.5'])
               + '; RMSE= %.3f ' % mean_squared_error(df['actual'], df['0.5'], squared=False)
               )
       #plt.text(1, 4, r'$R^2$=%.2f, RMSE=%.3f ' % (r2_score(df_actual_Qpred['actual'], df_actual_Qpred[0.5]),mean_squared_error(df_actual_Qpred['actual'], df_actual_Qpred[0.5], squared=False)))
    plt.grid(b=None)
    plt.show()
    plt.savefig(path + str(name) + '_PI pred.png', format='png', dpi=2000)

